## Visuals

In [17]:
import altair as alt
import altair_saver
import intake
import numpy as np
import pandas as pd
import shared_utils
from calitp import *
from shared_utils import altair_utils
from shared_utils import styleguide
from siuba import *

In [2]:
parquet = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/grants/BlackCat_TIRCP_DLA.parquet",
    engine="auto",
)

In [11]:
parquet.head()

,locode,name,BC_TIRCP_DLA_merge
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Only
1,6330.0,Willow Creek Community Services District,DLA Only
2,5036.0,Trinidad,DLA Only
3,5049.0,Ukiah,DLA Only
4,5082.0,Willits,DLA Only


In [23]:
parquet.name.nunique()

1217

In [27]:
summary = (
    parquet.BC_TIRCP_DLA_merge.value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "Grant_Platform",
                    "BC_TIRCP_DLA_merge": "Count_of_Organizations"})
)

In [28]:
summary

,Grant_Platform,Count_of_Organizations
0,DLA Only,991
1,BlackCat Only,168
2,BlackCat and DLA,33
3,TIRCP_Only,21
4,"TIRCP, BlackCat and DLA",17
5,TIRCP and BlackCat,1


In [24]:
# Basic bar chart: input df, x column, y column, column for color code, and chart title.
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word

def basic_bar_chart(df, x_col, y_col, color_col, chart_title=""):

    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            # column = "payment:N",
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(labeling(color_col))),
            ),
        )
        .properties(title=chart_title)
    )

    chart = styleguide.preset_chart_config(chart)

    return chart

In [29]:
basic_bar_chart(summary, 'Grant_Platform','Count_of_Organizations','Grant_Platform', 'Count of Organizations by Platform')

alt.Chart(...)